In [1]:
%matplotlib inline

In [2]:
import fipy as fp

In [3]:
from matplotlibElPhFViewer import MatplotlibElPhFViewer
def viewElPhF(filename, L="3.2 nm", nx=1200):
    L = fp.Variable(L)
    dx = L / nx
    mesh = fp.Grid1D(nx=nx, dx=dx/L)

    xi = fp.CellVariable(mesh=mesh, name=r"$\xi$")
    phi = fp.CellVariable(mesh=mesh, name=r"$\phi$")  
    interstitials = [
        fp.CellVariable(mesh=mesh, name="$e^-$")
    ]

    substitutionals = [
        fp.CellVariable(mesh=mesh, name="$M^+$"),
        fp.CellVariable(mesh=mesh, name="$A^-$")
    ]

    N = fp.CellVariable(mesh=mesh, name="$N$")
    
    components = interstitials + substitutionals + [N]
    
    for j in components:
        j.Y = fp.CellVariable(mesh=mesh, name=j.name)

    interstitials[0].z   = -1
    substitutionals[0].z = +2
    substitutionals[1].z = -2
    N.z                  = 0

                         
    values = fp.numerix.loadtxt(filename, skiprows=1, unpack=True)
    
    xi.setValue(values[1])
    phi.setValue(values[2])
    
    _start = 3
    for j, mu in zip(interstitials + substitutionals,
                     values[_start:_start+len(interstitials + substitutionals)]):
        j.setValue(mu)

    _start = 3 + len(interstitials + substitutionals)
    for j, Y in zip(components,
                     values[_start:_start+len(components)]):
        j.Y.setValue(Y)

    charge = fp.CellVariable(mesh=mesh, value=N.z)
    for i in interstitials + substitutionals:
        charge += i.z * i.Y

    viewer = MatplotlibElPhFViewer(phase=xi,
                           potential=phi,
                           components=[i.Y for i in components],
                           charge=charge,
                           potentials=[i - i[...,0] for i in interstitials + substitutionals],
                           limits={
                               'phasemax':1.,
                               'phasemin':0.,
                           })


## elphf0226.py

In [ ]:
!python elphf0226.py --galvani_potential="0.0 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="0.0_V.tsv" 

In [ ]:
viewElPhF("0.0_V.tsv")

In [ ]:
!python elphf0226.py --galvani_potential="0.5 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="0.5_V.tsv" 

In [ ]:
viewElPhF("0.5_V.tsv")

In [ ]:
!python elphf0226.py --galvani_potential="-0.1 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="-0.1_V.tsv" 

In [ ]:
viewElPhF("-0.1_V.tsv")

Negative Galvani potentials do not converge well. Prone to negative electron concentration, as a result of negative $Y_m$

In [ ]:
!python elphf0226.py --galvani_potential="-0.2 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="-0.2_V.tsv" 

In [ ]:
viewElPhF("-0.2_V.tsv")

Restart from previous solution at different Galvani potential does not result in "correct" equilibrium. Stabilizes at something with non-uniform electrochemical potentials. Possibly because equations are not well-coupled?

In [ ]:
!python elphf0226.py --galvani_potential="-0.2 V" --ncells=1200 --relaxation=0.8 --outer_sweeps=10 --output="-0.2_Vb.tsv" --restart="0.0_V.tsv" 

In [ ]:
viewElPhF("-0.2_Vb.tsv")

In [ ]:
"{0} {1} {2}".format(1.23, 3.14, 5)

In [ ]:
!python elphf0226.py params.json

In [4]:
%%bash

smt configure --executable=python
smt configure --labelgenerator=uuid
smt configure --on-changed=store-diff
smt configure --addlabel=parameters


Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly


In [ ]:
!mkdir Data

In [ ]:
!ls

In [ ]:
!smt run --main=elphf0226.py params.json

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,2):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l"

In [ ]:
import json
import pandas

!smt export
with open(".smt/records_export.json") as ff:
    data = json.load(ff)
    
df = pandas.DataFrame(data)

In [ ]:
df[["label","outcome","parameters"]]

In [ ]:
df.keys()

In [ ]:
!ls Data

In [ ]:
!ls Data/ce503af2e3b1/

In [ ]:
!cat Data/ce503af2e3b1/test.txt

In [ ]:
for r in df[["parameters"]].iterrows():
    print r[1]["parameters"]

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.01 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.10 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=1200 concentration="10.0 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=800 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=600 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=300 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=120 concentration="0.25 mol/l"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,20):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0226.py params.json galvani_potential="$galvani V" ncells=12000 concentration="0.25 mol/l"

In [ ]:
import timeit

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0237" 

Multiple versions found, using /data/ame2/miniconda/envs/fipy/bin/python. If you wish to use a different version, please specify it explicitly
L = 3.2 nm
barrier 145.295286758
$e^-$ 1.99957273847e-09 1.99999999308
$M^+$ 0.0045 0.99999999673
$A^-$ 0.00449999900021 1.87483286232e-10
$N$ 0.991000001 3.08279723394e-09
$m$ 1.000000002 2.99999999308
deltaV -0.693132797286
$e^-$ -20.7234651093 -20.7234651093
$M^+$ -4.00837154399 -4.00837154399
$A^-$ -6.78090295531 -6.78090295531
dt 0.0732421875
0.0 0.655212210675 -0.00471784553694
0.0 0.543689759255 -0.00660166924017
0.0 0.552850170925 -0.00719504322247
0.0 0.540128215863 -0.0070729211874
0.0 0.543682956284 -0.00712762262758
0.0 0.542219556953 -0.00710820426324
0.0 0.542758395425 -0.00711567157808
0.0 0.542552244663 -0.00711285014152
0.0 0.542600613786 -0.00711351483371
Record label for this run: '0b6231452b1b'
Data keys are [test.txt(4b78d72eea82d91984b96bd344cfaa5319ecad6e [2016-07-13 11:24:16]), output.tsv(b8b7b09909c51212de880f4056b85bc36

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0237.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0237"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0238"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0238.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0238"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0239"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0239.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0239"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0240.py params.json galvani_potential="$galvani V" ncells=1200 concentration="0.25 mol/l" --tag"0240"

In [ ]:
for galvani in fp.numerix.linspace(0,0.5,100):
    galvanistring="{0} V".format(galvani)
    !smt run --main=elphf0240.py params.json galvani_potential="$galvani V" ncells=1200 concentration="1.00 mol/l" --tag"0240"